# Make-o-Matic Gesture Recognition

## Part 3: Machine Learning

2017 by Thomas Lidy, TU Wien

### Requirements

Python 2.7

pip install -r requirements.txt

Tested on OS: Ubuntu 16.04.3 LTS

In [395]:
import numpy as np
import pandas as pd
import json
import time # for time measuring
import datetime # for time printing

from scipy import stats
from scipy.signal import resample
from collections import Counter # for majority vote
from collections import OrderedDict # for color palette

# Machine Learning
from sklearn import preprocessing, svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [396]:
def str_to_int(string):
    '''cut away first character and convert to int - used to convert Gesture IDs like "G01" to 1'''
    return int(string[1:])

In [397]:
def timestr(seconds):
    ''' returns HH:MM:ss formatted time string for given seconds
    (seconds can be a float with milliseconds included, but only the integer part will be used)
    :return: string
    '''
    return str(datetime.timedelta(seconds=int(seconds)))

## Read Meta-Data

In [398]:
# main data

# original input
#csv_file = 'data/EXPORT_09042017173622.csv'

# preprocessed input
csv_file = 'data/EXPORT_09042017173622_preprocessed.csv'


# json files to translate gestures, parcours into long text
#gestures_file = 'data/gestures.json' # this is the file edited manually by us to conform to json
gestures_file = 'data/gestures.json.orig' # this is the file edited manually by us to conform to json
parcours_file = 'data/parcours.json'
mutations_file = 'data/mutations.json'

files = (gestures_file, parcours_file, mutations_file)
dataframes = []

# NOTE THAT THESE JSON FILES ARE NOT JSON CONFORM
# each line is a json string on its own, so we need to process the json line by line and combine THEN into a list

In [399]:
def get_oid(oid_dict):
    # get from the original representation {u'$oid': u'589c8ed31337b5ab1e1be121'} just the oid
    return oid_dict['$oid']

In [400]:
# get meta-files with descriptions of gestures, parcours and mutations
for filename in files:
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in f]   # .decode("utf-8")

    lines = [json.loads(line) for line in lines]
    
    # convert list of json lines into Dataframe
    df = pd.DataFrame.from_dict(lines)
    
    # convert long $oid to short
    df['_id'] = df['_id'].apply(get_oid)
    
    # set the real id
    df.set_index('id', inplace=True)
    
    # convert index (ID) from string like 'G01' to int
    df.index = df.index.map(str_to_int)
    
    dataframes.append(df)

In [401]:
(gestures_df, parcours_df, mutations_df) = tuple(dataframes)

In [402]:
gestures_df

,_id,isGarbage,isNesture,name,slug
1,58a23a22d826756404709446,NaN,NaN,Single Rotation klein rechtsrum,rssr
2,58a23a22d826756404709447,NaN,NaN,Single Rotation klein linksrum,rssl
3,58a23a22d826756404709448,NaN,NaN,Oszillierende Rotation klein rechtsrum,rosr
4,58a23a22d826756404709449,NaN,NaN,Oszillierende Rotation klein linksrum,rosl
5,58a23a22d82675640470944a,NaN,NaN,Single Rotation groß rechtsrum,rsbr
6,58a23a22d82675640470944b,NaN,NaN,Single Rotation groß linksrum,rsbl
7,58a23a22d82675640470944c,NaN,NaN,Oszillierende Rotation groß rechtsrum,robr
8,58a23a22d82675640470944d,NaN,NaN,Oszillierende Rotation groß linksrum,robl
9,58a23a22d82675640470944e,NaN,NaN,Kontinuierliche Rotation groß rechtsrum,rcbr
10,58a23a22d82675640470944f,NaN,NaN,Kontinuierliche Rotation groß linksrum,rcbl


In [403]:
# "positive" gestures to recognize (not nestures)
gestures_pos = gestures_df[gestures_df['isNesture'] != True].index.tolist()
gestures_pos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [404]:
# "negative" gestures (nestures)
gestures_neg = gestures_df[gestures_df['isNesture'] == True].index.tolist()
nestures = gestures_neg # synonym
gestures_neg

[14, 15, 16, 17, 18]

#### Define handy function shortcut

In [405]:
def gesture_name(gesture_id):
    if gesture_id is None: return None
    return gestures_df.loc[gesture_id,'name']

## Read Experiment Data

In [406]:
# Experiment Data
data = pd.read_csv(csv_file)
data.head(10)

,Trainset,Experiment,Subject,TimeStamp,RFID,GRASP_A,GRASP_B,GRASP_C,AX,AY,AZ,EX,EY,EZ,Parcours,Parcours_Step,Mutation,Host,Host/Spot,Gesture
0,_TRAINSET14022017094616,1,Andreas,0,000000000000,781,8,797,0.06000,-0.02000,-0.10000,216.81250,9.06250,-81.93750,101,1,151,8,NaN,15
1,_TRAINSET14022017094616,1,Andreas,29001,000000000000,782,0,799,0.09000,-0.04000,-0.11000,217.06250,9.06250,-81.93750,101,1,151,8,NaN,15
2,_TRAINSET14022017094616,1,Andreas,46136,000000000000,782,6,798,0.12000,-0.09000,0.09000,217.43750,9.12500,-81.87500,101,1,151,8,NaN,15
3,_TRAINSET14022017094616,1,Andreas,74902,000000000000,784,7,798,0.08000,-0.08000,0.03000,217.62500,9.12500,-81.81250,101,1,151,8,NaN,15
4,_TRAINSET14022017094616,1,Andreas,97663,000000000000,781,0,798,0.07000,-0.09000,0.04000,217.93750,9.18750,-81.75000,101,1,151,8,NaN,15
5,_TRAINSET14022017094616,1,Andreas,116448,000000000000,784,4,800,0.12000,-0.06000,-0.03000,218.31250,9.25000,-81.75000,101,1,151,8,NaN,15
6,_TRAINSET14022017094616,1,Andreas,148753,000000000000,783,0,798,0.21000,-0.04000,0.03000,218.50000,9.31250,-81.75000,101,1,151,8,NaN,15
7,_TRAINSET14022017094616,1,Andreas,167422,000000000000,784,2,798,0.18000,-0.10000,-0.08000,218.68750,9.37500,-81.75000,101,1,151,8,NaN,15
8,_TRAINSET14022017094616,1,Andreas,187481,000000000000,782,4,799,0.15000,-0.18000,-0.03000,219.00000,9.43750,-81.75000,101,1,151,8,NaN,15
9,_TRAINSET14022017094616,1,Andreas,213733,000000000000,784,13,799,0.15000,-0.18000,-0.17000,219.12500,9.43750,-81.75000,101,1,151,8,NaN,15


In [407]:
# get the TimeStamp Diffs 
# (TimeStamps are reset after each Parcours, so we have to do it groupwise by Parcours)
group_by = ('Subject','Experiment','Trainset','Parcours')
timestamp_deltas = data.groupby(group_by)['TimeStamp'].diff()
#timestamp_deltas

In [408]:
timestamp_deltas.describe()

count     710,821.00000
mean       25,869.89066
std        35,094.26755
min           193.00000
25%        17,574.00000
50%        24,005.00000
75%        28,635.00000
max     5,743,871.00000
dtype: float64

In [409]:
# Note: Not sure why there is a value of 5,743,871 as the max timestamp delta

### Median Time Stamp delta

In [410]:
time_delta = timestamp_deltas.median()
print "Time stamp deltas are on median", time_delta, "micro-seconds"

Time stamp deltas are on median 24005.0 micro-seconds


In [411]:
time_delta_sec = time_delta / 1000000  # microsec to sec
time_delta_sec

0.024005

### Sampling Rate

In [412]:
sampling_rate = 1 /  time_delta_sec   
print "Signal sampling rate is on avg.", sampling_rate, "Hz"

Signal sampling rate is on avg. 41.6579879192 Hz


### Iterate through the data

In [413]:
# see gestures per Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours
Alfred   2           _TRAINSET14022017144824  101              [15, 1, 17]
                     _TRAINSET14022017144923  102              [15, 2, 17]
                     _TRAINSET14022017145122  103               [15, 1, 2]
                     _TRAINSET14022017145237  104               [15, 2, 1]
                     _TRAINSET14022017145434  107              [15, 1, 17]
                     _TRAINSET14022017145514  108              [15, 2, 17]
                     _TRAINSET14022017145629  109               [15, 1, 2]
                     _TRAINSET14022017145751  110               [15, 2, 1]
                     _TRAINSET14022017145913  113              [15, 1, 17]
                     _TRAINSET14022017145944  114              [15, 2, 17]
                     _TRAINSET14022017150026  115               [15, 1, 2]
                     _TRAINSET14022017150110  116               [15, 2, 1]
                     _TRAINSET14022017150614 

In [414]:
# see gestures per Parcours Step
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours  Parcours_Step
Alfred   2           _TRAINSET14022017144824  101       1                [15]
                                                        2                 [1]
                                                        3                [17]
                                                        4                 [1]
                                                        5                [17]
                                                        6                 [1]
                                                        7                [17]
                                                        8                 [1]
                                                        9                [17]
                                                        10                [1]
                     _TRAINSET14022017144923  102       1                [15]
                                                        2               

## Replace Nestures

In [415]:
replace_nestures = True

In [416]:
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Mutation','Gesture')
group_df = data.groupby(group_by)
print "Originally", len(group_df), "individual gesture blocks"

Originally 5169 individual gesture blocks


In [417]:
data.groupby(group_by).count().head(20)

TimeStamp  \
Subject Experiment Trainset                Parcours Parcours_Step Mutation Gesture              
Alfred  2          _TRAINSET14022017144824 101      1             151      15             142   
                                                    2             101      1               72   
                                                    3             152      17             102   
                                                    4             101      1               57   
                                                    5             152      17              77   
                                                    6             101      1               49   
                                                    7             152      17              66   
                                                    8             101      1               46   
                                                    9             152      17              65   
                                                    10            101      1               44   
                   _TRAINSET14022017144923 102      1             151      15              88   
                                                    2             102      2               54   
                                                    3             153      17              74   
                                                    4             102      2               37   
                                                    5             153      17              61   
                                                    6             102      2               41   
                                                    7             153      17              50   
                                                    8             102      2               35   
                                                    9             151      15              54   
                                                    10            102      2               37   

                                                                                    RFID  \
Subject Experiment Trainset                Parcours Parcours_Step Mutation Gesture         
Alfred  2          _TRAINSET14022017144824 101      1             151      15        142   
                                                    2             101      1          72   
                                                    3             152      17        102   
                                                    4             101      1          57   
                                                    5             152      17         77   
                                                    6             101      1          49   
                                                    7             152      17         66   
                                                    8             101      1          46   
                                                    9             152      17         65   
                                                    10            101      1          44   
                   _TRAINSET14022017144923 102      1             151      15         88   
                                                    2             102      2          54   
                                                    3             153      17         74   
                                                    4             102      2          37   
                                                    5             153      17         61   
                                                    6             102      2          41   
                                                    7             153      17         50   
                                                    8             102      2          35   
                                                    9             151      15         54   
                 

In [418]:
# Therefore Group by PARCOURS
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')

In [419]:
# Step 1: replace ALL Nestures by NaN
if replace_nestures:
    # make a copy of the complete data before altering anything
    data_nonest = data.copy()
    idx_nestures = data_nonest['Gesture'].isin(nestures)
    # replace nestures by NaN
    data_nonest.loc[idx_nestures,'Gesture'] = np.nan
    print data_nonest.head()

                  Trainset  Experiment  Subject  TimeStamp          RFID  \
0  _TRAINSET14022017094616           1  Andreas          0  000000000000   
1  _TRAINSET14022017094616           1  Andreas      29001  000000000000   
2  _TRAINSET14022017094616           1  Andreas      46136  000000000000   
3  _TRAINSET14022017094616           1  Andreas      74902  000000000000   
4  _TRAINSET14022017094616           1  Andreas      97663  000000000000   

   GRASP_A  GRASP_B  GRASP_C      AX       AY       AZ        EX      EY  \
0      781        8      797 0.06000 -0.02000 -0.10000 216.81250 9.06250   
1      782        0      799 0.09000 -0.04000 -0.11000 217.06250 9.06250   
2      782        6      798 0.12000 -0.09000  0.09000 217.43750 9.12500   
3      784        7      798 0.08000 -0.08000  0.03000 217.62500 9.12500   
4      781        0      798 0.07000 -0.09000  0.04000 217.93750 9.18750   

         EZ  Parcours  Parcours_Step  Mutation  Host Host/Spot  Gesture  
0 -81.93750 

In [420]:
# now we can use the Forward FILL and Backward FILL methods of Pandas
# to replace the NaNs by the values that come before or after

# BUT: we shall not do that across Parcours/Experiments!

In [421]:
# GROUPBY helps us here to apply the fill methods only within a PARCOURS

if replace_nestures:
    # BACKWARD FILL first by later values to NaNs before
    data_nonest = data_nonest.groupby(group_by).bfill()

    # in case there would be NaNs left, do also a FORWARD FILL
    #data = data.groupby(group_by).ffill()
    
    print "Replaced Nestures by filling with neighboured Gestures!"
    print np.isnan(data_nonest['Gesture']).sum(), "NaN values remaining. Should be 0."
    # NOTE: bfill applies to ALL COLUMNS! so there might be other columns affected by this!
    # TODO double-check any side effects!
    
    # adding NaNs cause the Gesture column to be converted from int to float
    # we convert back to int

    data_nonest['Gesture'] = data_nonest['Gesture'].astype(int)

Replaced Nestures by filling with neighboured Gestures!
0 NaN values remaining. Should be 0.


In [422]:
# check via groupby:
group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
group_df = data_nonest.groupby(group_by)
print "After nesture replacement", len(group_df), "individual gesture blocks"

After nesture replacement 720 individual gesture blocks


In [423]:
group_df.count()

TimeStamp  RFID  \
Subject Experiment Trainset                Parcours Gesture                    
Alfred  2          _TRAINSET14022017144824 101      1              720   720   
                   _TRAINSET14022017144923 102      2              531   531   
                   _TRAINSET14022017145122 103      1              458   458   
                                                    2              282   282   
                   _TRAINSET14022017145237 104      1              325   325   
                                                    2              476   476   
                   _TRAINSET14022017145434 107      1              576   576   
                   _TRAINSET14022017145514 108      2              518   518   
                   _TRAINSET14022017145629 109      1              292   292   
                                                    2              234   234   
                   _TRAINSET14022017145751 110      1              320   320   
                                                    2              379   379   
                   _TRAINSET14022017145913 113      1              558   558   
                   _TRAINSET14022017145944 114      2              541   541   
                   _TRAINSET14022017150026 115      1              322   322   
                                                    2              297   297   
                   _TRAINSET14022017150110 116      1              319   319   
                                                    2              472   472   
                   _TRAINSET14022017150614 401      5              353   353   
                                                    6              557   557   
                   _TRAINSET14022017150702 402      5              337   337   
                                                    6              439   439   
                   _TRAINSET14022017150748 403      5              397   397   
                                                    6              674   674   
                   _TRAINSET14022017151539 201      1              935   935   
                   _TRAINSET14022017151631 202      2              828   828   
                   _TRAINSET14022017151826 203      1              532   532   
                                                    2              226   226   
                   _TRAINSET14022017151945 206      1              803   803   
                   _TRAINSET14022017152030 207      2              807   807   
...                                                                ...   ...   
Dominik 7          _TRAINSET18022017103813 724      12            1813  1813   
                   _TRAINSET18022017105413 626      7             3445  3445   
                                                    8             2658  2658   
                   _TRAINSET18022017105840 627      7             2850  2850   
                                                    8             2777  2777   
                   _TRAINSET18022017110223 628      7             2873  2873   
                                                    8             1823  1823   
                   _TRAINSET18022017110459 629      7             2390  2390   
                                                    8             2346  2346   
                   _TRAINSET18022017110836 630      7             2819  2819   
                                                    8             1809  1809   
                   _TRAINSET18022017111316 331      3             1606  1606   
                                                    4             1383  1383   
                   _TRAINSET18022017111515 332      3             1702  1702   
                                                    4             1365  1365   
                   _TRAINSET18022017111659 333      3             1559  1559   
                                                    4             1322  1322   
                   _TRAINSET18022017111827 334      3        

In [424]:
# keep original data in a variable
data_orig = data

In [425]:
# from here on we use data again for data_nonest

if replace_nestures:
    data = data_nonest

## Data Pre-Procssing Part I

### Which Sensor Parameters to use?

In [426]:
include_GRASP = True

if include_GRASP:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ', 'GRASP_A', 'GRASP_B', 'GRASP_C']
else:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ']

# TODO add RFID?

### Global Normalize?

#### Normalize Parameter columns to -1, 1

here it's done globally. if set to False, there is an option to do it locally later

In [427]:
normalize_global = False
# normalize_global means we normalize all parameter columns at once, globally => NO LATER TREATMENT

In [428]:
data[params].head()

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.06000,-0.02000,-0.10000,216.81250,9.06250,-81.93750,781,8,797
1,0.09000,-0.04000,-0.11000,217.06250,9.06250,-81.93750,782,0,799
2,0.12000,-0.09000,0.09000,217.43750,9.12500,-81.87500,782,6,798
3,0.08000,-0.08000,0.03000,217.62500,9.12500,-81.81250,784,7,798
4,0.07000,-0.09000,0.04000,217.93750,9.18750,-81.75000,781,0,798


In [429]:
if normalize_global:
    # normalize to -1, 1
    data[params] = preprocessing.minmax_scale(data[params], feature_range=(-1, 1), axis=0, copy=False)

In [430]:
data[params].head()

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.06000,-0.02000,-0.10000,216.81250,9.06250,-81.93750,781,8,797
1,0.09000,-0.04000,-0.11000,217.06250,9.06250,-81.93750,782,0,799
2,0.12000,-0.09000,0.09000,217.43750,9.12500,-81.87500,782,6,798
3,0.08000,-0.08000,0.03000,217.62500,9.12500,-81.81250,784,7,798
4,0.07000,-0.09000,0.04000,217.93750,9.18750,-81.75000,781,0,798


## Get Isolated Gestures

### Grouping for each Gesture:

### Select correct level of detail:
* Parcours: all same gestures of a Parcours will be concatenated together
* Parcours-Step: gestures inside one Parcours will be keep individual

In [431]:
# select: 'Parcours' or 'Parcours_Step':
level_of_detail = 'Parcours'
#level_of_detail = 'Parcours-Step' 

In [432]:
# GET INDIVIDUAL GESTURES 
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours or Parcours-Step, Gesture

if level_of_detail == 'Parcours':
    group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
elif level_of_detail == 'Parcours-Step':
    group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Gesture')
else:
    raise ValueError("invalid level_of_detail")
    
group_df = data.groupby(group_by)
group_df.mean().head(100)  # mean is not meaningful here as aggregation - just to print the structure of the data

TimeStamp  \
Subject Experiment Trainset                Parcours Gesture                    
Alfred  2          _TRAINSET14022017144824 101      1        9,568,344.51667   
                   _TRAINSET14022017144923 102      2        7,422,825.57627   
                   _TRAINSET14022017145122 103      1        8,082,322.41048   
                                                    2       13,709,221.39362   
                   _TRAINSET14022017145237 104      1       14,005,074.11692   
                                                    2        9,481,434.69538   
                   _TRAINSET14022017145434 107      1        8,033,683.30208   
                   _TRAINSET14022017145514 108      2        7,227,369.08301   
                   _TRAINSET14022017145629 109      1        6,125,707.71575   
                                                    2        9,324,114.24786   
                   _TRAINSET14022017145751 110      1       12,223,926.44375   
                                                    2        7,817,470.07124   
                   _TRAINSET14022017145913 113      1        7,783,269.42115   
                   _TRAINSET14022017145944 114      2        7,577,012.02957   
                   _TRAINSET14022017150026 115      1        7,881,506.63665   
                                                    2        9,742,844.39731   
                   _TRAINSET14022017150110 116      1       13,022,919.87774   
                                                    2        9,031,461.86017   
                   _TRAINSET14022017150614 401      5       15,109,013.95184   
                                                    6       10,251,818.76481   
                   _TRAINSET14022017150702 402      5       12,762,223.20772   
                                                    6        8,697,379.23462   
                   _TRAINSET14022017150748 403      5       17,605,712.55919   
                                                    6       11,982,384.05193   
                   _TRAINSET14022017151539 201      1       12,229,690.33369   
                   _TRAINSET14022017151631 202      2       10,979,324.23309   
                   _TRAINSET14022017151826 203      1        8,344,590.61842   
                                                    2       14,676,017.94248   
                   _TRAINSET14022017151945 206      1       10,652,167.55168   
                   _TRAINSET14022017152030 207      2       10,735,968.12020   
...                                                                      ...   
                   _TRAINSET14022017162250 608      6       15,364,998.82331   
                   _TRAINSET14022017162537 611      5        8,942,319.51534   
                   _TRAINSET14022017162640 612      6        8,757,384.63191   
                   _TRAINSET14022017162748 613      5        6,786,025.36364   
                                                    6       11,712,726.60674   
                   _TRAINSET14022017162840 616      5       10,056,124.71162   
                   _TRAINSET14022017162927 617      6        9,321,950.65985   
                   _TRAINSET14022017163010 618      5        8,084,560.38735   
                                                    6       14,509,504.99592   
                   _TRAINSET14022017163318 621      5        9,656,076.57828   
                   _TRAINSET14022017163404 622      6        8,609,964.56390   
                   _TRAINSET14022017163453 623      5        6,342,932.38630   
                                                    6       11,087,419.31527   
                   _TRAINSET14022017163655 501      5        7,488,592.10512   
                                                    6       11,487,569.55482   
                   _TRAINSET14022017163905 502      5        8,195,221.68020   
                                                    6       11,722,859.65273   
                   _TRAINSET14022017164152 504      10       4,921,

In [433]:
print len(group_df), "individual gesture blocks"

720 individual gesture blocks


## Get Gesture Data: 1 Block per each individual Gesture

we put each time series that belong to 1 particular gesture in a particular parcours into a dictionary,
which contains a list of such time series blocks per gesture entry in the dict

### Create Gesture Dictionary + Reduce Data to desired parameter columns

In [434]:
# now we ITERATE nicely through group_df and get each Gesture block individually
# -> group_data will be a dataframe just for a single gesture

i=0
# dictionary containing a list of sub-datasets for each gesture, to train ML
gesture_exp_dict = {}

for name_tuple, datablock in group_df:
    i += 1
    #print str(name_tuple)
    gesture = name_tuple[-1]  # gesture is last element of tuple, as defined in group_by above
    
    # initalize empty list for the gesture if not existing in dict yet
    if gesture not in gesture_exp_dict.keys():
        gesture_exp_dict[gesture] = [] 
        
    # reduce dataframe to params columns
    datablock = datablock[params] 
    
    # add data to gesture dict
    gesture_exp_dict[gesture].append(datablock)
    
    # NOTE that group_data here still contains ALL data columns. we will redue to params later

print "DONE:", i, "gesture blocks"

DONE: 720 gesture blocks


#### Some Statistics about the Gesture Block data

In [435]:
# How many data blocks = training examples do we have for each gesture
for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, '\t', len(gesture_exp_dict[gest]), "training data blocks", '\t', gesture_name(gest) 

G 1 	120 training data blocks 	Single Rotation klein rechtsrum
G 2 	120 training data blocks 	Single Rotation klein linksrum
G 3 	81 training data blocks 	Oszillierende Rotation klein rechtsrum
G 4 	81 training data blocks 	Oszillierende Rotation klein linksrum
G 5 	60 training data blocks 	Single Rotation groß rechtsrum
G 6 	60 training data blocks 	Single Rotation groß linksrum
G 7 	20 training data blocks 	Oszillierende Rotation groß rechtsrum
G 8 	20 training data blocks 	Oszillierende Rotation groß linksrum
G 9 	20 training data blocks 	Kontinuierliche Rotation groß rechtsrum
G 10 	20 training data blocks 	Kontinuierliche Rotation groß linksrum
G 11 	66 training data blocks 	LinearMovement Single
G 12 	32 training data blocks 	LinearMovement Oszillierend
G 13 	20 training data blocks 	Drücken


In [436]:
# how many data points (= samples or timesteps) does each data block have?

data_sizes = {} # collect per gesture in dict
data_sizes_total = [] # collect all in list

print "Average length per gesture:"

for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, ':\t', 
    data_sizes[gest] = []
    for datablock in gesture_exp_dict[gest]:
        size = datablock.shape[0]
        data_sizes[gest].append(size)
        data_sizes_total.append(size)
    avg_gesture_len = int(np.mean(data_sizes[gest]))
    print "% d samples \t%0.2f sec" % (avg_gesture_len, avg_gesture_len * time_delta_sec)

Average length per gesture:
G 1 :	 650 samples 	15.60 sec
G 2 :	 577 samples 	13.85 sec
G 3 :	 1365 samples 	32.77 sec
G 4 :	 1302 samples 	31.25 sec
G 5 :	 687 samples 	16.49 sec
G 6 :	 561 samples 	13.47 sec
G 7 :	 2861 samples 	68.68 sec
G 8 :	 2413 samples 	57.92 sec
G 9 :	 464 samples 	11.14 sec
G 10 :	 466 samples 	11.19 sec
G 11 :	 1027 samples 	24.65 sec
G 12 :	 1965 samples 	47.17 sec
G 13 :	 905 samples 	21.72 sec


In [437]:
# minimum, average and maximum sample size of gestures
gest_len_min = min(data_sizes_total)
gest_len_avg = np.mean(data_sizes_total)
gest_len_max = max(data_sizes_total)

In [438]:
gest_len_min, gest_len_avg, gest_len_max

(168, 988.0, 4124)

In [439]:
# average data length (number of samples)
print "Average length all gestures"
print "% d samples \t%0.2f sec" % (gest_len_avg, gest_len_avg * time_delta_sec)    

Average length all gestures
 988 samples 	23.72 sec


In [440]:
pd.Series(data_sizes_total).describe()

count     720.00000
mean      988.00000
std       676.29644
min       168.00000
25%       489.75000
50%       760.00000
75%     1,328.50000
max     4,124.00000
dtype: float64

In [441]:
samples = int(gest_len_avg)

### Low-Pass Filter - Testing

removing high frequencies (little fluctuations which are probably not relevant)

In [443]:
# source code from https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    '''cutoff: cutoff frequency in Hz
    fs: sampling rate in Hz'''
    nyq = 0.5 * fs # Nyquist frequency is half the sampling rate (fs)
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [444]:
# Filter settings
#fs = 30.0       # sample rate, Hz
fs = sampling_rate   # determined before by average time delta 
# CHOOSE HERE desired cutoff frequency of the filter Hz
order = 1 #3 #5 #6

cutoff = 4 #Hz
#cutoff = 3.667 
#cutoff = 1.3
#cutoff = 0.667 
#cutoff = 0.5
#cutoff = 0.33

In [445]:
def preprocess_signal(testdata, 
                      normalize=False, 
                      resampling=False, n_samples=None, timestamps=None, window='hann', 
                      filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        
        if n_samples is None:
            # if not a FIXED number of samples is provided, the number of samples stays the same as in the input signal
            n_samples = testdata.shape[0] 
        
        if timestamps is None:
            testdata = resample(testdata, num=n_samples, window=window)
        else:
            # if provided, we use the original timestamps to re-align the signal
            # TODO check: n_samples must match len(timestamps)
            testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps, window='hann')
        

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

## Feature Calculation

### Zero Crossing Rate

In [446]:
def calc_zero_crossings(datablock, normalized=False):
    '''computes row-wise zerocrossings'''
    # datablock is assumed to be pandas Dataframe and to have multiple signals in the rows
    # example for 1 signal row:
    #zcr = np.signbit(signal).diff().abs().mean()
    # for multiple signal rows:
    zcr = np.signbit(datablock).astype(int).diff(axis=0).abs().mean(axis=0)
    
    if normalized:
        # divide by length of signal, otherwise it will be directly related to the size of the chosen window
        zcr = zcr / datablock.shape[0]
    return zcr

### Statistical Features

In [447]:
# Calc statistical features

def calc_statistical_features(matrix, axis=0):

    # to define the proper output shape, we need the "other axis" of the input shape (not the one where we compute along)
    other_axis = int(not axis) 
    n_rows = matrix.shape[other_axis]
    
    result = np.zeros((n_rows,7))
    
    result[:,0] = np.mean(matrix, axis=axis)
    result[:,1] = np.var(matrix, axis=axis, dtype=np.float64) 
    result[:,2] = stats.skew(matrix, axis=axis)
    result[:,3] = np.median(matrix, axis=axis)
    result[:,4] = np.min(matrix, axis=axis)
    result[:,5] = np.max(matrix, axis=axis)
    result[:,6] = stats.kurtosis(matrix, axis=axis, fisher=False) # Matlab calculates Pearson's Kurtosis

    result[np.where(np.isnan(result))] = 0
    return result

### Function to compute All features

In [448]:
def calc_all_features(in_data, calc_derivative=False, calc_zerocrossings=False):

    # calc statistical features
    features = calc_statistical_features(in_data, axis=0)

    # vectorize
    features = features.flatten()

    if calc_derivative:
        # calc derivative of all signals
        in_data_deriv = np.gradient(in_data, axis=0)
        # calc statistics of derivatives
        features_deriv = calc_statistical_features(in_data_deriv, axis=0)
        # vectorize
        features_deriv = features_deriv.flatten()
        # concatenate to other features
        features = np.concatenate((features,features_deriv))

    if calc_zerocrossings:
        features_zcr = calc_zero_crossings(in_data)
        features = np.concatenate((features,features_zcr))

    return features

## Start Feature Calculation

#### Set Options here:

In [449]:
# OPTIONS:

# either/or:
use_lowpassfilter = False
use_normalized = True 
use_resampled = True 
# if both are False, unresampled unnormalized input is used

# other options: # True is better for all
exclude_non_gestures = True
calc_derivative = True
calc_zerocrossings = True

In [450]:
# we added preprocess_signal() function below, thats why we need to use the original gesture_dict as input
input_dict = gesture_exp_dict 

In [451]:
if exclude_non_gestures:
    gestures_to_process = gestures_pos
else:
    gestures_to_process = input_dict.keys()

In [452]:
# COMPUTE FEATURES
# LOOP over all gesture data to create features

# initialize feature output for training data as a list
train_list = []
train_classes_num = []

for gest in sorted(gestures_to_process):
    print "G", gest, ':\t', len(input_dict[gest]), "examples"
    
    for in_data in input_dict[gest]:
        #print datablock.shape, 
        
        #if use_resampled:
        #    # resampled data has already extracted the param columns
        #    in_data = datablock
        #else:
        #    # for non-resampled we have to get the relevant data columns and transpose
        #    in_data = datablock[params].T
        
        # preprocessing
        in_data = preprocess_signal(in_data, use_normalized, 
                                    use_resampled, samples, timestamps=None, window=None, # 'hann'
                                    filtering=use_lowpassfilter)
                
        # convert to dataframe cause we use pandas .diff() in ZCR computation
        in_data = pd.DataFrame(in_data, columns=params)

        # calculate features
        features = calc_all_features(in_data, calc_derivative, calc_zerocrossings)

        # append to output list
        train_list.append(features)
        
        # store class (gesture number) for these features
        train_classes_num.append(gest)

G 1 :	120 examples
G 2 :	120 examples
G 3 :	81 examples
G 4 :	81 examples
G 5 :	60 examples
G 6 :	60 examples
G 7 :	20 examples
G 8 :	20 examples
G 9 :	20 examples
G 10 :	20 examples
G 11 :	66 examples
G 12 :	32 examples
G 13 :	20 examples


In [453]:
features.shape

(135,)

## Machine Learning

### Prepare Training Data

In [454]:
print "Training data:", len(train_list), "examples"

Training data: 720 examples


In [455]:
# make feature array from feature list (ALL training data)

train_data = np.array(train_list)
#del train_list
train_data.shape

(720, 135)

In [456]:
# verify if the training categories (gesture numbers) have the same length
len(train_classes_num)

720

### Standardize

Zero-mean unit-variance Standardization

In [457]:
# ad-hoc scaling
# train_data = preprocessing.scale(train_data,axis=0)
# axis=0 means independently standardize each feature, otherwise (if 1) standardize each sample

In [458]:
# we now user StandardScaler class to keep the mean and variance for later
standardizer = preprocessing.StandardScaler()
train_data = standardizer.fit_transform(train_data)

### Train/Test Set Split

In [459]:
# split the data into train/test set

testset_size = 0.25

# sklearn >= 0.18
# use random_state to avoid that the results fluctuate randomly
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0) 
splits = splitter.split(train_data, train_classes_num)

# Note: this for loop is only executed once, if n_splits==1
for train_index, test_index in splits:
    #print "TRAIN INDEX:", train_index
    #print "TEST INDEX:", test_index
    
    # split the data
    train_set = train_data[train_index]
    test_set = train_data[test_index]
    
    # and the numeric classes (groundtruth)
    train_classes = np.array(train_classes_num)[train_index]
    test_classes = np.array(train_classes_num)[test_index]
    
    print "TRAIN SIZE:", train_set.shape
    print "TEST SIZE:", test_set.shape
    

TRAIN SIZE: (540, 135)
TEST SIZE: (180, 135)


## 1) Gesture Regonition - isolated (+ independent of host)

### ML Algorithm: SVM

Support Vector Machines

In [460]:
# try 3 different SVM kernels
kernels = ['linear','poly','rbf']

In [461]:
models = {}

for kernel in kernels:
    print "SVM", kernel,
    
    # TRAIN 
    start_time = time.time() # measure time

    model = OneVsRestClassifier(SVC(kernel=kernel)) #, degree=degree)) #, n_jobs=-1)  # n_jobs = n cpus, -1 = all
    # full set
    #model.fit(train_data, train_classes_num)
    # train set
    model.fit(train_set, train_classes)
    
    # store in dict
    models[kernel] = model

    end_time = time.time()
    print "Training time:", timestr(end_time - start_time)

SVM linear Training time: 0:00:00
SVM poly Training time: 0:00:00
SVM rbf Training time: 0:00:00


#### Verification on Train Set (just for plausibility)

In [462]:
# predict on train set
pred_train = model.predict(train_set)
pred_train

array([ 6,  2,  1,  5,  9,  4, 10,  1,  5, 11,  3, 12,  4,  7, 12, 12,  2,
        2,  1,  1,  1,  6,  1,  6,  3, 11, 12,  3,  2,  4,  4,  1,  1,  9,
        4,  1,  1, 11, 11,  1, 11, 13,  4,  1,  2,  4,  2,  8,  6,  4, 10,
        6,  9,  5,  4,  5, 11,  7,  1,  5,  8,  3, 11,  6,  3, 11,  6, 11,
        6, 13,  1, 11,  3,  1, 11,  1, 11, 13,  5, 13,  3,  3,  1,  6,  1,
        1,  2,  8, 12,  2,  1,  2, 12,  1,  2,  3,  3,  4,  1, 10,  6, 11,
        2,  1,  4,  2, 11,  2,  2,  1,  5,  6,  3, 11,  6,  3,  1,  5, 12,
        9,  2,  1,  1,  2,  3,  2,  2,  4, 11,  2,  6,  4,  1, 11,  9,  1,
       12,  2,  8,  6,  1,  1, 11,  7, 11, 12,  5,  6, 11,  3,  1, 12, 10,
        3,  3,  2,  3,  5, 13,  4,  4,  4,  1,  2,  2,  6,  3,  5, 10,  5,
        2,  1,  1, 13, 11,  2, 12,  5,  1,  3, 11, 10, 11,  5,  6,  5,  3,
        1,  4,  3,  5,  6, 10,  4,  5,  3,  1,  9, 11,  2,  8,  6,  4,  3,
       11,  6,  6, 11,  2, 11,  4,  1,  1,  4,  2,  2, 12, 11,  9,  2,  7,
        6,  3, 11,  1,  1

In [463]:
train_classes

array([ 6,  2,  1,  5,  9,  4, 10,  1,  5, 11,  3, 12,  4,  7, 12, 12,  1,
        2,  1,  1,  1,  5,  1,  6,  3, 11, 12,  3,  2,  4,  4,  1,  1,  9,
        4,  1,  1, 11, 11,  1, 11, 13,  4,  1,  2,  4,  2,  8,  6,  4, 10,
        6,  9,  5,  4,  5, 11,  7,  1,  5,  8,  3, 11,  6,  3, 11,  6, 11,
        6, 13,  1, 11,  3,  1, 11,  1, 11, 13,  5, 13,  3,  3,  1,  6,  1,
        1,  2,  8, 12,  2,  1,  2, 12,  1,  2,  3,  3,  4,  1, 10,  6, 11,
        2,  1,  4,  2, 11,  2,  2,  1,  5,  6,  3, 11,  6,  3,  1,  5, 12,
        9,  2,  1,  1,  2,  3,  2,  2,  4, 11,  2,  6,  4,  1, 11,  9,  1,
       12,  2,  8,  6,  1,  1, 11,  7, 11, 12,  5,  6, 11,  3,  1, 12, 10,
        3,  3,  2,  3,  5, 13,  4,  4,  4,  1,  2,  2,  6,  3,  5, 10,  5,
        2,  1,  1, 13, 11,  2, 12,  5,  1,  3, 11, 10, 11,  5,  6,  5,  3,
        1,  4,  3,  5,  6, 10,  4,  5,  3,  1,  9, 11,  2,  8,  6,  4,  3,
       11,  6,  6, 11,  2, 11,  4,  1,  1,  4,  2,  2, 12, 11,  9,  2,  7,
        1,  3, 11,  1,  1

In [464]:
# Accuracy on train set (manual computation)
np.sum(pred_train == train_classes) * 1.0 / len(train_classes)

0.98703703703703705

In [465]:
# Accuracy on train set (using scikit-learn)
accuracy_score(train_classes, pred_train)

0.98703703703703705

## Evaluation

### Evaluation - Overall

In [466]:
result_ov = pd.DataFrame(index=kernels, columns=['Accuracy','Precision','Recall','F-Measure'])

In [467]:
for k in kernels:
    # predict on TEST set
    pred_test = models[k].predict(test_set) 
    
    # Accuracy, Precision, Reacall on TEST set
    result_ov.loc[k,'Accuracy'] = accuracy_score(test_classes, pred_test)
    result_ov.loc[k,'Precision'] = precision_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'Recall'] = recall_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'F-Measure'] = f1_score(test_classes, pred_test, average='macro')

In [468]:
pd.options.display.float_format = '{:,.2f}'.format
result_ov*100

,Accuracy,Precision,Recall,F-Measure
linear,70.00,74.94,72.49,73.09
poly,81.11,85.64,87.37,85.88
rbf,81.11,83.39,87.88,84.74


### Evaluation - Per Gesture

In [469]:
# manual selection which one was the best one
best_model = models['poly']
pred_test = best_model.predict(test_set) 

In [470]:
# TODO check if the sorting of precision_score etc. is really in this order!!
labels = sorted(np.unique(test_classes))
gesture_names = [gesture_name(l) for l in labels]

In [471]:
# nice result dataframe
columns = ['Gesture','N_train','N_test','Precision','Recall','F1']
result_df = pd.DataFrame(index=labels,columns=columns)
result_df['Gesture'] = gesture_names

In [472]:
# number of train / test instances
values, counts = np.unique(train_classes, return_counts=True)
result_df['N_train'] = pd.Series(counts, index=values)
values, counts = np.unique(test_classes, return_counts=True)
result_df['N_test'] = pd.Series(counts, index=values)

In [473]:
# per class evaluation
result_df['Precision'] = precision_score(test_classes, pred_test, average=None) * 100
result_df['Recall'] = recall_score(test_classes, pred_test, average=None) * 100
result_df['F1'] = f1_score(test_classes, pred_test, average=None) * 100

In [474]:
result_df

,Gesture,N_train,N_test,Precision,Recall,F1
1,Single Rotation klein rechtsrum,90,30,70.97,73.33,72.13
2,Single Rotation klein linksrum,90,30,78.26,60.00,67.92
3,Oszillierende Rotation klein rechtsrum,61,20,83.33,75.00,78.95
4,Oszillierende Rotation klein linksrum,61,20,86.96,100.00,93.02
5,Single Rotation groß rechtsrum,45,15,92.86,86.67,89.66
6,Single Rotation groß linksrum,45,15,90.91,66.67,76.92
7,Oszillierende Rotation groß rechtsrum,15,5,100.00,100.00,100.00
8,Oszillierende Rotation groß linksrum,15,5,80.00,80.00,80.00
9,Kontinuierliche Rotation groß rechtsrum,15,5,100.00,100.00,100.00
10,Kontinuierliche Rotation groß linksrum,15,5,83.33,100.00,90.91


In [475]:
# compare average P, R and F to overall P, R and F above (same)
result_df.mean(axis=0)

N_train     41.54
N_test      13.85
Precision   85.64
Recall      87.37
F1          85.88
dtype: float64

In [476]:
# Confusion Matrix
conf = confusion_matrix(test_classes, pred_test, labels=labels) # labels defines the order
labels_long = gestures_df.loc[labels,'name']
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df

,1,2,3,4,5,6,7,8,9,10,11,12,13
name,,,,,,,,,,,,,
Single Rotation klein rechtsrum,22,3,1,0,1,0,0,0,0,0,3,0,0
Single Rotation klein linksrum,7,18,2,0,0,1,0,0,0,1,1,0,0
Oszillierende Rotation klein rechtsrum,1,0,15,2,0,0,0,1,0,0,0,1,0
Oszillierende Rotation klein linksrum,0,0,0,20,0,0,0,0,0,0,0,0,0
Single Rotation groß rechtsrum,0,1,0,0,13,0,0,0,0,0,1,0,0
Single Rotation groß linksrum,1,1,0,0,0,10,0,0,0,0,3,0,0
Oszillierende Rotation groß rechtsrum,0,0,0,0,0,0,5,0,0,0,0,0,0
Oszillierende Rotation groß linksrum,0,0,0,1,0,0,0,4,0,0,0,0,0
Kontinuierliche Rotation groß rechtsrum,0,0,0,0,0,0,0,0,5,0,0,0,0


## 2) Continuous Time Series Prediction

What is our input stream?

The data of 1 trainset, because after each trainset, the TimeStamp is reset.

In [477]:
# a) loop over each Trainset
#group_by = ('Subject','Experiment','Trainset')

# b) use Experiment as the block where we do predictions (means it includes timestamp resets!!)
group_by = ('Subject','Experiment')

group_df = data.groupby(group_by)
group_df.max().head(50) 

Trainset  TimeStamp          RFID  GRASP_A  \
Subject Experiment                                                              
Alfred  2           _TRAINSET14022017172235   44132577  760057911BAB      880   
        8           _TRAINSET18022017151654  122720319  280015E55981      861   
Andreas 1           _TRAINSET14022017123316   48620325  760057911BAB      902   
        6           _TRAINSET17022017155347  152209477  09006734114B      895   
Claudia 3           _TRAINSET15022017120738   71354787  760057911BAB      855   
        5           _TRAINSET17022017115327  170698407  280015E55981      851   
Dominik 4           _TRAINSET15022017173325   68968234  760057911BAB      875   
        7           _TRAINSET18022017114128  147807390  09006734114B      882   

                    GRASP_B  GRASP_C    AX    AY    AZ     EX    EY     EZ  \
Subject Experiment                                                           
Alfred  2               825      870 15.86 10.51 14.77 359.94 89.56 179.88   
        8               833      858 29.66 18.07 21.64 359.94 88.25 158.31   
Andreas 1               825      900 37.08 30.30 14.35 359.94 89.56 179.94   
        6               848      896 30.62 25.67 19.96 359.94 89.75 179.94   
Claudia 3               757      847 26.03 22.74 13.31 359.94 89.88 179.94   
        5               763      836 30.86 20.59 27.93 359.94 89.88 179.94   
Dominik 4               783      877 15.59 19.41 20.36 359.94 89.56 179.94   
        7               829      879 34.37 21.63 22.44 359.94 89.56 179.94   

                    Parcours  Parcours_Step  Mutation  Host Host/Spot  Gesture  
Subject Experiment                                                              
Alfred  2                802             12       853   127         B       11  
        8                903             12       956   122         A       13  
Andreas 1                802             12       853   122         B       11  
        6                903             12       956   127         A       13  
Claudia 3                802             12       853   127         B       11  
        5                903             12       956   122         A       13  
Dominik 4                802             12       853   127         B       11  
        7                903             12       956   122         A       13

In [478]:
print len(group_df), "Experiments / Trainsets"

8 Experiments / Trainsets


In [479]:
# iterate over each Trainset
i =0
for name_tuple, group_data in group_df:
    i += 1
    #print str(name_tuple)
    
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    break # for testing we just do 1 loop
    

In [480]:
name_tuple

('Alfred', 2)

In [481]:
group_data['TimeStamp'].min()

0

In [482]:
group_data['TimeStamp'].max()

44132577

In [483]:
if len(name_tuple) == 3:
    # check if TimeStamps are monotonously increasing
    if not np.all(group_data['TimeStamp'].diff()[1:] > 0):
        raise ValueError("Time Stamps are not monotonously increasing!")

In [484]:
# set these to None so that plot title is not shown wrongly
parcours = None
mutation = None
gesture = None

In [485]:
# which gestures appear in this Experiment or Trainset
group_data['Gesture'].unique()

array([ 1,  2,  6,  5, 11, 10,  9])

### Pre-Process the Data - Testing

the same way as it was done for training set

In [486]:
pd.options.display.float_format = '{:,.5f}'.format

In [487]:
# get the relevant columns out of group_data

In [488]:
timestamps = group_data['TimeStamp'].tolist()

In [489]:
test_gestures = group_data['Gesture'].tolist()

In [490]:
# 9 parameters columns
testdata = group_data[params]
testdata.shape

(64432, 9)

In [491]:
# Global Min/max normalization
# Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
# (currently its done per training block) and the same scaling values (min and max) should be reused here
# see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# TODO store minmax_scale from training data and reapply same scaling here

if normalize_global:
    testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0, copy=make_copy)

In [492]:
# convert to dataframe cause plot needs column names
testdata = pd.DataFrame(testdata, columns=params)
testdata.head(15)

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
66713,0.07000,-0.05000,0.04000,229.31250,10.25000,-80.06250,759,677,768
66714,0.03000,-0.05000,0.08000,229.31250,10.25000,-80.06250,758,676,768
66715,-0.03000,-0.08000,-0.02000,229.25000,10.25000,-80.06250,758,675,767
66716,-0.09000,-0.13000,-0.02000,229.25000,10.25000,-80.06250,758,676,767
66717,-0.10000,-0.10000,-0.13000,229.25000,10.25000,-80.06250,759,676,769
66718,-0.04000,-0.02000,-0.04000,229.25000,10.25000,-80.06250,758,675,766
66719,0.00000,-0.06000,-0.08000,229.25000,10.25000,-80.06250,757,675,769
66720,0.03000,-0.03000,0.04000,229.31250,10.25000,-80.06250,759,676,770
66721,0.17000,-0.03000,0.03000,229.37500,10.25000,-80.12500,759,676,768
66722,0.19000,-0.08000,0.14000,229.37500,10.18750,-80.12500,758,675,768


In [493]:
# time resample

n_samples = len(timestamps)  

if use_resampled:
    # the number of samples stays the same
    # but we use the original timestamps to re-align the signal
    testdata_res, timestamps2 = resample(testdata, num=n_samples, t=timestamps)
    
    # convert to dataframe cause plot needs column names
    testdata_res = pd.DataFrame(testdata_res, columns=params)

In [494]:
timestamps[:15]

[0,
 24915,
 49032,
 74016,
 111220,
 122772,
 143717,
 172480,
 186423,
 211319,
 237536,
 261141,
 302690,
 305197,
 341003]

In [495]:
timestamps2[:15]

array([      0.,   24915.,   49830.,   74745.,   99660.,  124575.,
        149490.,  174405.,  199320.,  224235.,  249150.,  274065.,
        298980.,  323895.,  348810.])

In [496]:
# timestamps are now equidistant
timestamps2[1:15] - timestamps2[:14]

array([ 24915.,  24915.,  24915.,  24915.,  24915.,  24915.,  24915.,
        24915.,  24915.,  24915.,  24915.,  24915.,  24915.,  24915.])

In [497]:
testdata_res.head(15)

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.07000,-0.05000,0.04000,229.31250,10.25000,-80.06250,759.00000,677.00000,768.00000
1,0.03000,-0.05000,0.08000,229.31250,10.25000,-80.06250,758.00000,676.00000,768.00000
2,-0.03000,-0.08000,-0.02000,229.25000,10.25000,-80.06250,758.00000,675.00000,767.00000
3,-0.09000,-0.13000,-0.02000,229.25000,10.25000,-80.06250,758.00000,676.00000,767.00000
4,-0.10000,-0.10000,-0.13000,229.25000,10.25000,-80.06250,759.00000,676.00000,769.00000
5,-0.04000,-0.02000,-0.04000,229.25000,10.25000,-80.06250,758.00000,675.00000,766.00000
6,0.00000,-0.06000,-0.08000,229.25000,10.25000,-80.06250,757.00000,675.00000,769.00000
7,0.03000,-0.03000,0.04000,229.31250,10.25000,-80.06250,759.00000,676.00000,770.00000
8,0.17000,-0.03000,0.03000,229.37500,10.25000,-80.12500,759.00000,676.00000,768.00000
9,0.19000,-0.08000,0.14000,229.37500,10.18750,-80.12500,758.00000,675.00000,768.00000


In [498]:
# debug check whether the values have been altered -> OK
#testdata == testdata_res

In [499]:
# overwrite testdata with testdata_res for subsequent coherent usage
#testdata = testdata_res

### Continuous Prediction

In [500]:
# for our window_size (= signal length of input to Machine Learning)
# we take the average signal length of the trained gestures
window_size = gest_len_avg # gest_len_min 
window_size

988.0

In [501]:
# PREDICTION RESOLUTION
# how quickly do we step forward

# for now we choose half the window_size
step_size = window_size / 2

# can be set smaller for higher resolution

# TODO: set in milliseconds - convert back to sample length

step_size

494.0

In [502]:
# TODO: align with preprocess_signal function used in training data above

def preprocess_signal_continuous(testdata, normalize=False, resampling=False, timestamps=None, filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        # the number of samples stays the same
        # if provided, we use the original timestamps to re-align the signal
        n_samples = testdata.shape[0] # must match len(timestamps)
        testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps) #, window='hann')

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

In [503]:
# PREDICTION LOOP OVER 1 TRAINING INPUT BLOCK

def continuous_prediction(testdata, window_size, step_size):
    pos = 0
    n_samples = testdata.shape[0]
    
    # output
    test_groundtruth = [] # we create the groundtruth to compare with here
    predictions = []  # predictions are collected here

    while pos < (n_samples - window_size):
        # cut a window out of the incoming signal
        signal = testdata[pos:pos+window_size]

        # to get the "correct" gesture for that window, we cut the same part of the gesture information
        test_window_groundtruth = test_gestures[pos:pos+window_size]

        # we do a majority vote to say which gesture is pre-dominant in this window
        gt_gesture = Counter(test_window_groundtruth).most_common()[0][0]

        # calc features
        features = calc_all_features(signal, calc_derivative, calc_zerocrossings)

        # reshape to row vector for standardize and predict below (= single input sample)
        features = features.reshape(1, -1)  
        
        # STANDARDIZE features, the same way as done in training (reusing those mean and var)
        features = standardizer.transform(features)

        # ML prediction of gesture
        pred_gesture = best_model.predict(features)[0]

        # add to groundtruth and prediction list
        test_groundtruth.append(gt_gesture)
        predictions.append(pred_gesture)

        # step forward
        pos += step_size
    
    return test_groundtruth, predictions

In [504]:
# LOOP over ALL Experiments or Trainsets

i = 0
n_groups = len(group_df)

test_groundtruth_all = [] # we create the groundtruth to compare with here
predictions_all = []  # predictions are collected here

for name_tuple, group_data in group_df:
    
    i += 1
    print "Experiment", i, "/", n_groups, ":", str(name_tuple), group_data.shape,
    
    # just metadata
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    # get signals, timestamps and gesture groundtruth
    timestamps = group_data['TimeStamp'].tolist()
    test_gestures = group_data['Gesture'].tolist()
    testdata = group_data[params]
    
    # preprocess testdata
    print "Preprocessing ...",
    testdata = preprocess_signal_continuous(testdata, use_normalized, use_resampled, timestamps, use_lowpassfilter)
    #print testdata.shape
    
    # convert to dataframe cause we use pandas .diff() in ZCR computation
    testdata = pd.DataFrame(testdata, columns=params)
    
    print "Prediction:", 
    test_groundtruth, predictions = continuous_prediction(testdata, window_size, step_size)
    print len(predictions), "predictions"
    
    test_groundtruth_all.extend(test_groundtruth)
    predictions_all.extend(predictions)
    

Experiment 1 / 8 : ('Alfred', 2) (64432, 20) Preprocessing ... Prediction:

TypeError: cannot do slice indexing on <class 'pandas.indexes.range.RangeIndex'> with these indexers [988.0] of <type 'numpy.float64'>

In [ ]:
n_samples

In [ ]:
gesture_name(11)

In [ ]:
features.shape

In [ ]:
n_samples

In [ ]:
print len(predictions_all), "predictions"

In [ ]:
print "collected true gestures include:"
np.unique(test_groundtruth_all).tolist()

In [ ]:
print "predicted gestures include:"
np.unique(predictions_all).tolist()

In [ ]:
pd.DataFrame({'groundt':test_groundtruth_all, 'pred':predictions_all})

In [ ]:
result_ov = pd.DataFrame(columns=['result']) #columns=['Accuracy','Precision','Recall','F-Measure'])

# Accuracy, Precision, Reacall on TEST set
result_ov.loc['Accuracy'] = accuracy_score(test_groundtruth_all, predictions_all)
result_ov.loc['Precision'] = precision_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['Recall'] = recall_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['F-Measure'] = f1_score(test_groundtruth_all, predictions_all, average='macro')
result_ov

#### Confusion Matrix

In [ ]:
conf = confusion_matrix(test_groundtruth_all, predictions_all, labels=labels) # labels defines the order

In [ ]:
labels_long = gestures_df.loc[labels,'name']
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df

In [ ]:
labels_long